# Visualize Flow Decompositions

In this notebook we look at the example of the unfolding of the (3, 4, 13) triangle.

We deform this surface, find *interesting* directions for flow decompositions on this surface and then rescale to produce a nice presentation of the surface for these flow decompositions.

Note that this notebook is mentioned as a reference in the Article [A new Orbit Closure in Genus 8](https://arxiv.org/abs/2110.05407). This notebook describes the techniques that were used to obtain the picture of the surface in this article. The "proof" of the statement of Theorem A.1 can be found in [this section](#Relations-between-Edges) of the notebook.

## Unfold a Polygon to create a Translation Surface

In [ ]:
from ipyvue_flatsurf import Widget
from flatsurf import translation_surfaces, polygons, similarity_surfaces

In [ ]:
t = polygons.triangle(3, 4, 13)
B = similarity_surfaces.billiard(t)
S = B.minimal_cover('translation')
S = S.erase_marked_points()

## Visulation of a Flow Decomposition

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(S)
D = O.decomposition((1, 0))

In [ ]:
V = Widget(D)
print(D)
V

## Use the Same Glueings for Another Flow Decomposition
Clicking on half edges in the above figure forces edges to be glued. The same gluings will then be replayed in the figure below.

#### At the moment, this is not supported. (TODO)

```py
import ipywidgets

DD = O.decomposition((0, 1))
print(DD)
W = Widget(DD)
ipywidgets.link((V,'inner'), (W, 'forced'))
W
```

# Passing to a Deformed Surface

We deform the surface with a vector from the tangent space so that this is not a billiard anymore.

In [ ]:
for decomposition in O.decompositions(64):
    O.update_tangent_space_from_flow_decomposition(decomposition)
    if O.dimension() > 2: break

tangent = O.lift(O.tangent_space_basis()[2])
length = sum(abs(x.parent().number_field(x)) for x in tangent) / len(tangent)

upper_bound = 1
while upper_bound < length:
    upper_bound *= 2
    
# Shrink the deformation somewhat more so we do not need too many flips.
upper_bound *= 2

print(f"Deforming with a tangent of length approximately {upper_bound}")

In [ ]:
from flatsurf.geometry.pyflatsurf_conversion import from_pyflatsurf

try:
    deformation = [O.V2(x / upper_bound, x / (2*upper_bound)).vector for x in tangent]
    deformed = from_pyflatsurf((O._surface + deformation).codomain()).delaunay_triangulation()
except Exception:
    print("The deformation crosses over a vertex. Trying with a smaller deformation.")
    upper_bound *= 2
    
deformed

## Search for Interesting Directions to Visualize

Good directions might be ones where cylinders do not intersect much. To find such directions, we search for a direction with a cylinder and a minimal component, and among those, one with short cylinders.

In [ ]:
O = GL2ROrbitClosure(deformed)

v0 = None
height = None

for decomposition in O.decompositions_breadth_first(2):
    print(".", end="")
    if len(decomposition.cylinders()) == 0:
        continue
    if len(decomposition.minimal_components()) == 0:
        continue
        
    h = max(cylinder.height() for cylinder in decomposition.cylinders())
    if height is None or h < height:
        v0 = decomposition.u
        height = h
        print(f"Found {decomposition} whose cylinders are shorter than {h}")

In [ ]:
D = O.decomposition(v0)
print(D)
Widget(D.cylinders())

Find a second direction which decomposes differently and whose cylinders do not intersect the cylinders of the first direction.

In [ ]:
T = D.decomposition.triangulation()
OT = GL2ROrbitClosure(T)
DT = OT.decomposition(v0)

perimeter = [p.saddleConnection().source() for cylinder in DT.cylinders() for p in cylinder.perimeter() if p.vertical()]
for p in perimeter[:]:
    perimeter.append(-p)
perimeter = set(perimeter)

In [ ]:
v1 = None
intersections = None
height = None

for direction in GL2ROrbitClosure(deformed).decompositions_breadth_first(2):
    print(".", end="")
    if direction.u == v0:
        continue

    if len(direction.cylinders()) == 0:
        continue
    if len(direction.minimal_components()) == 0:
        continue

    decomposition = OT.decomposition(direction.u)
    
    if len(decomposition.cylinders()) == len(D.cylinders()) and len(decomposition.minimal_components()) == len(D.minimal_components()):
        pass
    else:
        OO = GL2ROrbitClosure(deformed)
        OO.update_tangent_space_from_flow_decomposition(D)
        dim = OO.dimension()
        OO.update_tangent_space_from_flow_decomposition(direction)
        if OO.dimension() == dim:
            # This direction decomposes like the first one and it does not add anything to the tangent space.
            continue

    i = len([c for cylinder in decomposition.cylinders() for p in cylinder.perimeter() if p.vertical() for c in p.saddleConnection().crossings() if c in perimeter])
    if intersections is None or i < intersections:
        h = max(cylinder.height() for cylinder in decomposition.cylinders())
        if height is None or h < height:
            intersections = i
            height = h
            v1 = direction.u
            print(f"Found {decomposition} whose cylinders are shorter than {h} and intersect {i} times with the other cylinders.")

In [ ]:
DD = O.decomposition(v1)
print(DD)
Widget(DD.cylinders())

We can also show all cylinders in the same picture:

In [ ]:
Widget(D.cylinders() + DD.cylinders())

## Making the Search Directions Orthogonal

In [ ]:
A = matrix([v0, v1])
E = matrix([[1, 0], [0, 1]])
N = E * ~A.transpose()

In [ ]:
T = deformed.apply_matrix(N, in_place=False)
T = T.delaunay_triangulation()

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(T)
D = O.decomposition(E[0])
DD = O.decomposition(E[1])
W = Widget(D.cylinders() + DD.cylinders())
W

## Relations between Edges
Theorem A.1 of "A new Orbit Closure in Genus 8" states that some relations hold between the edges in the above picture.

In [ ]:
W.labels = 'NUMERIC'

from flatsurf.geometry.pyflatsurf_conversion import to_pyflatsurf
S = to_pyflatsurf(T)

We check for the relevant edges that they are related as annouced:

In [ ]:
def relate(a, b):
    a = S.fromHalfEdge(int(a))
    b = S.fromHalfEdge(int(b))
    assert a.x() * b.y() == a.y() * b.x()
    return a.x() / b.x() if b.x() else a.y() / b.y()

In [ ]:
print(relate(1, -26)) # U and V
print(relate(14, 16)) # C and D
print(relate(2, -12)) # B and A
print(relate(22, -7)) # S and T

### Passing to a Quotient
Actually, that Theorem A.1 makes a statement about a quotient of that picture. That quotient can be determined as follows:

In [ ]:
def filter_map(a, b):
    if a == 1R:
        return b == -29R
    return True

isomorphism = S.isomorphism(S, filter_matrix=lambda a, b, c, d: True, filter_map=filter_map)
assert isomorphism.has_value()

In [ ]:
isomorphism.value()

TODO: Visualize the surface module the quotient here again.

## Make Cylinders Aligned with Triangulation

Pass to a triangulation that is aligned with the cylinders.

In [ ]:
T = from_pyflatsurf(DD.decomposition.triangulation())
T

Currently, the minimal components are triangulated in a way that respects their internal structure, see https://github.com/flatsurf/flatsurf/issues/272. This leads to very stretched out pictures.
We Delaunay triangulate the interior of such components to make these components shorter.

In [ ]:
O = GL2ROrbitClosure(T)
D = O.decomposition(E[1])
S = D.decomposition.surface()
noncylinders = D.minimal_components() + D.undetermined_components()

inners = []

for component in D.decomposition.components():
    is_perimeter = lambda p: (component.cylinder() and p.vertical()) or (not component.cylinder() and not p.boundary())
    perimeter = [connection.saddleConnection().source() for connection in component.perimeter() if is_perimeter(connection)]
    
    for p in component.perimeter():
        connection = p.saddleConnection()
        if connection != type(connection)(S, connection.source()):
            print("Boundary of component not aligned with triangulation. Is this a bug in libflatsurf?")
        inner = connection.source()
        if not is_perimeter(p):
            inners.append(inner)
            inners.append(-inner)
        while True:
            inner = S.nextAtVertex(inner)
            if inner in perimeter or -inner in perimeter:
                break
            inners.append(inner)
            inners.append(-inner)
    
while True:
    for inner in inners:
        import pyflatsurf
        if S.delaunay(inner) == 0:
            S.flip(inner)
            break
    else:
        break
        
T = from_pyflatsurf(S)
T

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(T)
D = O.decomposition(E[0])
DD = O.decomposition(E[1])
print(D)
print(DD)
Widget(D.cylinders() + DD.cylinders())

Now the pictures might be very squeezed, so try to rescale to make things reasonable again.

In [ ]:
ratios = [float(c.width() / c.height()) for c in D.cylinders()] + [float(c.height() / c.width()) for c in DD.cylinders()]
print(f"Old aspect ratio of cylinders: {ratios}")

# We try to move everything simultaneously close to the same aspect ratio.
target = float(1 / 1)

# The factor we would have to multiply each cylinder with to make it there.
distance = [target / ratio for ratio in ratios]

# We average over these factors.
scale = exp(sum([log(d) for d in distance]) / len(distance))

# Turn the above factor into something that lives in the number field.
scale = ZZ(ceil(scale * 1000)) / 1000
ratios = [scale * ratio for ratio in ratios]

print(f"New aspect ratio of cylinders: {ratios}")
A = matrix([[scale, 0], [0, 1]])
U = T.apply_matrix(A, in_place=False)

Glue cylinders in horizontal direction. At the moment this is not implemented anymore. (TODO)

In [ ]:
O = GL2ROrbitClosure(U)
D = O.decomposition(E[0])
DD = O.decomposition(E[1])
print(D)
V = Widget(D.cylinders() + DD.cylinders())

forced = []

surface = D.decomposition.surface()
for cylinder in D.cylinders():
    for connection in cylinder.perimeter():
        connection = connection.saddleConnection()
        assert connection == type(connection)(surface, connection.source())
        
    perimeter = [connection.saddleConnection().source() for connection in cylinder.perimeter()]
    inners = []
    
    for connection in cylinder.perimeter():
        inner = connection.saddleConnection().source()
        if not connection.vertical():
            inners.append(inner)
        while True:
            inner = surface.nextAtVertex(inner)
            if inner in perimeter or -inner in perimeter:
                break
            inners.append(inner)

    # Force all inner edges except for the one least aligned to the flow direction.
    def dot(he):
        v = surface.fromHalfEdge(he)
        return abs(float(v.x()) * float(E[0][0]) + float(v.y()) * float(E[0][1]))
        
    smallest = inners[0]
    
    for inner in inners:
        if dot(inner) < dot(smallest):
            smallest = inner

    forced.extend(list(set([he.id() for he in inners if he != smallest and he != -smallest])))
        
V.forced = forced
V

### Do these directions tell us about the full orbit closure?

In [ ]:
print(O)

In [ ]:
O.update_tangent_space_from_flow_decomposition(D)
print(O)

In [ ]:
O.update_tangent_space_from_flow_decomposition(DD)
print(O)